Эта тетрадка из размеченных eaf файлов создаёт библейскую часть датасета

In [1]:
import os, re
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import warnings
from bs4 import XMLParsedAsHTMLWarning
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning, module='bs4')
from pydub import AudioSegment
import mutagen 
from mutagen.wave import WAVE 

In [2]:
eaf_filenames = sorted([name for name in os.listdir("../biblio") if name.endswith('_ann.eaf')])
eaf_filenames

['001 vki onmain_ann.eaf',
 '002 vki nonap bylbu odaan_ann.eaf',
 '003 byl vkiduk rritne_ann.eaf',
 '004 byl kenilikki guukenten_ann.eaf',
 '005 vki trre mv tilkamkandaan dugulin_ann.eaf',
 '006 byl trre oyalbudaatna nan vkiv omodaatnan_ann.eaf',
 '007 vki byldu nan-da ichuunche_ann.eaf',
 '008 vki avraam tedemen yaruttan_ann.eaf',
 '009 iisus khristos baldadakan_ann.eaf',
 '010 miami oldov epkudek_ann.eaf',
 '011 iisus nan samariya ain_ann.eaf',
 '012 iisus tunon tikicha byu debukendeen_ann.eaf',
 '013 iisus bylbu bgdedden_ann.eaf',
 '014 iisus egden unev odukanin_ann.eaf',
 '015 arikalbu ilbeendeen_ann.eaf',
 '016 iisus lazaru urukanin_ann.eaf',
 '017 iisus bylbu upkuchedden_ann.eaf',
 '018 iisus mutt u kelmettey emnin_ann.eaf',
 '019 iisus bylbu ayday emnin_ann.eaf',
 '020 iisus noman miavandukur mendeschirilbu mentiki gadin_ann.eaf',
 '021 iisus kkedeken_ann.eaf',
 '022 iisus urrin_ann.eaf',
 '023 iisus nanindula ennin_ann.eaf',
 '024 iisustu tederil chakabakan baldanmain_ann.eaf',

In [3]:
# Получаем soup файла
def get_soup(filename):
    with open('../biblio/'+filename) as f:
        file = f.read()
    soup = BeautifulSoup(file)
    return soup

In [4]:
# Получаем таймкоды границ
def get_timecodes(soup):
    timecodes = soup.find_all('time_slot')
    ts_msec = {}
    for timecode in timecodes:
        if timecode.get('time_value'):
            ts_msec[timecode.get('time_slot_id')] = int(timecode.get('time_value'))
    return ts_msec

In [53]:
file_phrase_start_end = []
for eaf_filename in tqdm(eaf_filenames):
    soup = get_soup(eaf_filename)
    ts_msec = get_timecodes(soup)
    
    for tier in soup.find_all('tier'):
        anns = tier.find_all('alignable_annotation')
        for i, ann in enumerate(anns):
            ann_ts_1 = ann.get('time_slot_ref1')
            ann_ts_2 = ann.get('time_slot_ref2')
            phrase = ann.find('annotation_value').text

            ann_num = '000'[:-len(str(i+1))]+str(i+1)
            if eaf_filename.startswith('pr'):
                filename = 'eve_biblical_pr_'+eaf_filename[3:6]+'_'+ann_num+'.wav'
            else:
                filename = 'eve_biblical_'+eaf_filename[:3]+'_'+ann_num+'.wav'

            newAudio = AudioSegment.from_wav("../biblio/"+eaf_filename[:-8]+'.wav')
            newAudio = newAudio[ts_msec[ann_ts_1]:ts_msec[ann_ts_2]]
            newAudio.export('../dataset/'+filename, format="wav")
            
            file_phrase_start_end.append([filename,
                                          phrase, 
                                          ts_msec[ann_ts_1],
                                          ts_msec[ann_ts_2]])

100%|███████████████████████████████████████████| 29/29 [00:22<00:00,  1.30it/s]


In [56]:
df = pd.DataFrame(file_phrase_start_end, columns = ['file', 'text', 'start', 'end'])
df[['file', 'text']].to_csv('even_speech_biblical.csv', index=False)

### Немного статистики

In [26]:
df = pd.DataFrame(file_phrase_start_end, columns = ['file', 'text', 'start', 'end'])
df['time_len'] = df['end']-df['start']
df.head()

,file,text,start,end,time_len
0,eve_biblical_001_1.wav,Хөвки Дукундукун укчэнэкэл,1040,3470,2430
1,eve_biblical_001_2.wav,Хөвки онмаин,5370,6477,1107
2,eve_biblical_001_3.wav,Нонан Хөвки няним нян төру оча.,7880,10710,2830
3,eve_biblical_001_4.wav,"Тараком төррэ як-та ачча бичэ,",11486,13700,2214
4,eve_biblical_001_5.wav,хатарсинюн мөнюн бичэл.,14100,15560,1460


**Столько звучащих минут**

In [31]:
df['time_len'].sum() / 60000

67.9605

**Столько минут было в аудио (с тишиной)**

In [39]:
filenames = os.listdir('../biblio')
whole_l = 0
for filename in tqdm(filenames):
    if filename.endswith('wav'):
        audio = WAVE("../biblio/"+filename)
        audio_info = audio.info
        length = int(audio_info.length)
        whole_l += length
print(whole_l / 60)

100%|███████████████████████████████████████| 120/120 [00:00<00:00, 1368.97it/s]

89.26666666666667
